In [1]:
# from kafka import KafkaConsumer
# from json import loads

In [2]:
# consumer = KafkaConsumer(
#     'BTC',
#     bootstrap_servers=['2072befba3b8:9092'],
#     auto_offset_reset='earliest',
#     enable_auto_commit=True,
#     group_id='my-group',
#     value_deserializer=lambda x: loads(x.decode('utf-8'))
# )

In [3]:
# for mes in consumer:
#     mes = mes.value
#     print(mes)

In [4]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/spark-streaming-kafka-0-8-assembly_2.11-2.0.0-preview.jar pyspark-shell' 

In [10]:
# from pyspark import SparkContext
# from pyspark.streaming import StreamingContext
# # from pyspark.streaming.kafka import KafkaUtils
# # import json

# # sc  = SparkContext(appName="TestSpark")
# # ssc = StreamingContext(sc, 10)

# # kafkaStream = KafkaUtils.createStream(ssc, '2072befba3b8:9092', 'spark-streaming', {"BTC":1})
# # parsed.count().map(lambda x: "Batch: %s" %x).pprint()

# # ssc.start()
# # ssc.awaitTermination()


from pyspark import SparkContext                                                                                        
from pyspark.sql import SparkSession                                                                                    
from pyspark.streaming import StreamingContext                                                                          
from pyspark.streaming.kafka import KafkaUtils    
import pandas as pd
import json
import yaml

ModuleNotFoundError: No module named 'pyspark.streaming.kafka'